# BigDataProgramming TermProject

## 엔카 크롤링

### Driver Load

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import json

In [10]:
driver = webdriver.Chrome(service= Service(ChromeDriverManager().install()))

In [11]:
# 페이지 로딩 후 대기 시간 설정
wait_time = 1

### run
#### 매물 조회 리스트를 순회하며 무작위 `매물id` 수집
##### 중복되는 `매물id` 는 제외

In [12]:
# 페이지 순회를 위한 범위 지정 (예: 1~5페이지)
page_start = 1
page_end = 15  # 원하는 페이지 수까지 변경 가능

# 전체 추출 번호를 저장할 리스트
all_numbers = set()

for page in range(page_start, page_end + 1):
    try:
        # 페이지 URL 업데이트
        url = f"https://car.encar.com/list/car?page={page}"
        driver.get(url)
        
        # 페이지 로딩 대기
        time.sleep(wait_time)  # 필요에 따라 WebDriverWait으로 변경 가능
        
        # data-impression 값 추출
        index = 1
        while True:
            try:
                xpath = f"/html/body/div/div[1]/div[4]/div[1]/div[{index}]"
                element = driver.find_element(By.XPATH, xpath)
                data_impression = element.get_attribute("data-impression")
                if data_impression:
                    # 맨 앞 번호만 추출
                    first_number = data_impression.split("|")[0]
                    all_numbers.add(first_number)  # 번호 누적 저장
                index += 1
            except:
                # 현재 페이지의 매물 끝
                break
        
        print(f"페이지 {page} 크롤링 완료, 현재까지 추출된 번호 수: {len(all_numbers)}")
    except Exception as e:
        print(f"페이지 {page}에서 오류 발생: {e}")

# 최종 결과 출력
print("모든 페이지에서 추출된 번호들:")
for number in all_numbers:
    print(number)


페이지 1 크롤링 완료, 현재까지 추출된 번호 수: 10
페이지 2 크롤링 완료, 현재까지 추출된 번호 수: 20
페이지 3 크롤링 완료, 현재까지 추출된 번호 수: 30
페이지 4 크롤링 완료, 현재까지 추출된 번호 수: 40
페이지 5 크롤링 완료, 현재까지 추출된 번호 수: 50
페이지 6 크롤링 완료, 현재까지 추출된 번호 수: 60
페이지 7 크롤링 완료, 현재까지 추출된 번호 수: 70
페이지 8 크롤링 완료, 현재까지 추출된 번호 수: 80
페이지 9 크롤링 완료, 현재까지 추출된 번호 수: 90
페이지 10 크롤링 완료, 현재까지 추출된 번호 수: 100
페이지 11 크롤링 완료, 현재까지 추출된 번호 수: 110
페이지 12 크롤링 완료, 현재까지 추출된 번호 수: 119
페이지 13 크롤링 완료, 현재까지 추출된 번호 수: 129
페이지 14 크롤링 완료, 현재까지 추출된 번호 수: 139
페이지 15 크롤링 완료, 현재까지 추출된 번호 수: 149
모든 페이지에서 추출된 번호들:
38597547
38451325
38317702
38150050
38620988
37625760
38139996
38083238
38313249
37385150
38579790
38255080
38426365
38380772
38047373
38188192
38595557
38558206
38617845
38344511
38535566
38555433
37100694
38210208
38424930
38078840
38280296
38207811
38434587
38613063
38588096
38563299
38409191
38337990
38043251
38485703
38220888
38535177
38437499
38601963
38087184
38521387
38491651
38597986
38415883
37309699
38571217
38608765
37994188
38581336
38549520
38590266
38501564
38380889
3821

### run
#### 수집한 `매물id` 를 `https://fem.encar.com/cars/report/accident/{매물id}` 에 대입해 보험이력 조회

In [13]:
try:
    for car_id in all_numbers:
        # 4. URL 접속
        url = f"https://fem.encar.com/cars/report/accident/{car_id}"
        driver.get(url)
        
        # 페이지 로딩 대기
        time.sleep(wait_time)  # 필요시 WebDriverWait으로 대체 가능
        
        # 5. 데이터 추출
        try:
            print("매물id:", car_id)
            elements = driver.find_elements(By.XPATH, "/html/body/div[1]/div/div[2]/div[2]/div[1]/ul/li[1]/span")
            car_detail = ",".join([element.text for element in elements])
            print("일반사양:", car_detail)
            print("용도변경이력:", driver.find_element(By.XPATH, "//li[contains(.,'용도 변경이력')]//span").text)
            print("번호소유자변경이력:", driver.find_element(By.XPATH, "//li[contains(.,'번호 / 소유자 변경이력')]//span").text)
            print("특수사고이력:", driver.find_element(By.XPATH, "//li[contains(.,'특수 사고이력')]//span").text)
            print("보험사고_내차:", driver.find_element(By.XPATH, "//div/div[2]/div[2]/div[1]/ul/li[5]/span").text)
            print("보험사고_타차:", driver.find_element(By.XPATH, "//div/div[2]/div[2]/div[1]/ul/li[6]/span").text)

        finally:
            print()
        #     # 기본 정보
        #     일반사양 = driver.find_element(By.CLASS_NAME, "ReportAccidentSummary_txt__fVCew").text
        #     용도변경이력 = driver.find_element(By.XPATH, "//li[contains(.,'용도 변경이력')]//span").text
        #     번호소유자변경이력 = driver.find_element(By.XPATH, "//li[contains(.,'번호 / 소유자 변경이력')]//span").text
        #     특수사고이력 = driver.find_element(By.XPATH, "//li[contains(.,'특수 사고이력')]//span").text
        #     보험사고_내차 = driver.find_element(By.XPATH, "//li[contains(.,'보험사고 이력 (내차 피해)')]//span").text
        #     보험사고_타차 = driver.find_element(By.XPATH, "//li[contains(.,'보험사고 이력 (타차 가해)')]//span").text
            
        #     # 차량 스펙
        #     제작사 = driver.find_element(By.XPATH, "//th[contains(.,'제작사')]/following-sibling::td").text
        #     자동차명 = driver.find_element(By.XPATH, "//th[contains(.,'자동차명')]/following-sibling::td").text
        #     연식 = driver.find_element(By.XPATH, "//th[contains(.,'연식')]/following-sibling::td").text
        #     사용연료 = driver.find_element(By.XPATH, "//th[contains(.,'사용연료')]/following-sibling::td").text
        #     배기량 = driver.find_element(By.XPATH, "//th[contains(.,'배기량')]/following-sibling::td").text
        #     차체형상 = driver.find_element(By.XPATH, "//th[contains(.,'차체형상')]/following-sibling::td").text
        #     용도차종 = driver.find_element(By.XPATH, "//th[contains(.,'용도 및 차종')]/following-sibling::td").text
        #     최초보험가입일 = driver.find_element(By.XPATH, "//th[contains(.,'최초 보험 가입일자')]/following-sibling::td").text

        #     # 사용 이력
        #     영업용대여 = driver.find_element(By.XPATH, "//th[contains(.,'영업용(대여) 사용이력')]/following-sibling::td").text
        #     영업용일반 = driver.find_element(By.XPATH, "//th[contains(.,'영업용(일반) 사용이력')]/following-sibling::td").text
        #     관용용도 = driver.find_element(By.XPATH, "//th[contains(.,'관용용도 사용이력')]/following-sibling::td").text

        #     # 보험사고
        #     전손보험사고 = driver.find_element(By.XPATH, "//th[contains(.,'전손 보험사고')]/following-sibling::td").text
        #     침수보험사고 = driver.find_element(By.XPATH, "//th[contains(.,'침수 보험사고')]/following-sibling::td").text
        #     도난보험사고 = driver.find_element(By.XPATH, "//th[contains(.,'도난 보험사고')]/following-sibling::td").text

        #     # 6. 데이터 저장
        #     data.append({
        #         "매물등록번호": car_id,
        #         "일반사양": 일반사양,
        #         "용도변경이력": 용도변경이력,
        #         "번호/소유자변경이력": 번호소유자변경이력,
        #         "특수사고이력": 특수사고이력,
        #         "보험사고이력(내차피해)": 보험사고_내차,
        #         "보험사고이력(타차가해)": 보험사고_타차,
        #         "제작사": 제작사,
        #         "자동차명": 자동차명,
        #         "연식": 연식,
        #         "사용연료": 사용연료,
        #         "배기량": 배기량,
        #         "차체형상": 차체형상,
        #         "용도 및 차종": 용도차종,
        #         "최초 보험 가입일자": 최초보험가입일,
        #         "영업용(대여)사용이력": 영업용대여,
        #         "영업용(일반)사용이력": 영업용일반,
        #         "관용용도사용이력": 관용용도,
        #         "전손보험사고": 전손보험사고,
        #         "침수보험사고": 침수보험사고,
        #         "도난보험사고": 도난보험사고
        #     })
        # except Exception as e:
        #     print(f"ID {car_id}에서 데이터를 추출하는 중 오류 발생: {e}")

finally:
    # 7. 브라우저 닫기
    # driver.quit()
    print("finished")

매물id: 38597547
일반사양: 기아더 뉴 레이2020년식박스형
용도변경이력: 사용이력있음
번호소유자변경이력: 1회3회
특수사고이력: 전손 0회침수(전손,분손) 0회도난 0회
보험사고_내차: 없음
보험사고_타차: 없음

매물id: 38451325
일반사양: 벤츠A-클래스 W1772020년식해치백
용도변경이력: 없음
번호소유자변경이력: 0회0회
특수사고이력: 전손 0회침수(전손,분손) 0회도난 0회
보험사고_내차: 1회914,490원
보험사고_타차: 없음

매물id: 38317702
일반사양: BMW7시리즈 (G70)2024년식세단
용도변경이력: 없음
번호소유자변경이력: 0회1회
특수사고이력: 전손 0회침수(전손,분손) 0회도난 0회
보험사고_내차: 없음
보험사고_타차: 없음

매물id: 38150050
일반사양: 벤츠E-클래스 W2132022년식세단 4도어
용도변경이력: 없음
번호소유자변경이력: 0회4회
특수사고이력: 전손 0회침수(전손,분손) 0회도난 0회
보험사고_내차: 2회17,816,546원
보험사고_타차: 2회/4,348,040원

매물id: 38620988
일반사양: 

finished


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//li[contains(.,'용도 변경이력')]//span"}
  (Session info: chrome=131.0.6778.86); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x011033E3+25059]
	(No symbol) [0x0108CDE4]
	(No symbol) [0x00F6BEC3]
	(No symbol) [0x00FAFD86]
	(No symbol) [0x00FAFFCB]
	(No symbol) [0x00FED952]
	(No symbol) [0x00FD1F44]
	(No symbol) [0x00FEB51E]
	(No symbol) [0x00FD1C96]
	(No symbol) [0x00FA3FAC]
	(No symbol) [0x00FA4F3D]
	GetHandleVerifier [0x013F5543+3113795]
	GetHandleVerifier [0x0140A20A+3198986]
	GetHandleVerifier [0x014029E2+3168226]
	GetHandleVerifier [0x011A3250+680016]
	(No symbol) [0x0109572D]
	(No symbol) [0x010929D8]
	(No symbol) [0x01092B75]
	(No symbol) [0x010857D0]
	BaseThreadInitThunk [0x76B07BA9+25]
	RtlInitializeExceptionChain [0x77A4C0CB+107]
	RtlClearBits [0x

#### test

In [14]:
try:
    # 2. 테스트용 URL 설정
    maemul_id = "38554502"  # 매물 ID
    url = f"https://fem.encar.com/cars/report/accident/{maemul_id}"
    driver.get(url)

    # 3. 첫 번째 섹션 데이터 추출
    xpath1 = "/html/body/div[1]/div/div[2]/div[2]/div[1]"
    section1 = WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, xpath1)))
    section1_text = section1.text

    # 4. 두 번째 섹션 데이터 추출
    xpath2 = "/html/body/div[1]/div/div[2]/div[2]/div[2]"
    section2 = WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, xpath2)))
    section2_text = section2.text

    # 5. 결과 출력
    # print("첫 번째 섹션 데이터:")
    # print(section1_text)
    # print("\n두 번째 섹션 데이터:")
    # print(section2_text)

    # 기본 정보
    elements = driver.find_elements(By.XPATH, "/html/body/div[1]/div/div[2]/div[2]/div[1]/ul/li[1]/span")
    car_detail = ",".join([element.text for element in elements])
    print("일반사양:", car_detail)
    print("용도변경이력:", driver.find_element(By.XPATH, "//li[contains(.,'용도 변경이력')]//span").text)
    print("번호소유자변경이력:", driver.find_element(By.XPATH, "//li[contains(.,'번호 / 소유자 변경이력')]//span").text)
    print("특수사고이력:", driver.find_element(By.XPATH, "//li[contains(.,'특수 사고이력')]//span").text)
    print("보험사고_내차:", driver.find_element(By.XPATH, "//div/div[2]/div[2]/div[1]/ul/li[5]/span").text)
    print("보험사고_타차:", driver.find_element(By.XPATH, "//div/div[2]/div[2]/div[1]/ul/li[6]/span").text)
    
    # 차량 스펙
    print("제작사:", driver.find_element(By.XPATH, "//th[contains(.,'제작사')]/following-sibling::td").text)
    print("자동차명:", driver.find_element(By.XPATH, "//th[contains(.,'자동차명')]/following-sibling::td").text)
    print("연식:", driver.find_element(By.XPATH, "//th[contains(.,'연식')]/following-sibling::td").text)
    print("사용연료:", driver.find_element(By.XPATH, "//th[contains(.,'사용연료')]/following-sibling::td").text)
    print("배기량:", driver.find_element(By.XPATH, "//th[contains(.,'배기량')]/following-sibling::td").text)
    print("차체형상:", driver.find_element(By.XPATH, "//th[contains(.,'차체형상')]/following-sibling::td").text)
    # print("용도차종:", driver.find_element(By.XPATH, "//th[contains(.,'용도 및 차종')]/following-sibling::td").text)
    # print("최초보험가입일:", driver.find_element(By.XPATH, "//th[contains(.,'최초 보험 가입일자')]/following-sibling::td").text)
    
    # 사용 이력
    print("영업용대여:", driver.find_element(By.XPATH, "//th[contains(.,'영업용(대여) 사용이력')]/following-sibling::td").text)
    print("영업용일반:", driver.find_element(By.XPATH, "//th[contains(.,'영업용(일반) 사용이력')]/following-sibling::td").text)
    print("관용용도:", driver.find_element(By.XPATH, "//th[contains(.,'관용용도 사용이력')]/following-sibling::td").text)
    
    # 보험사고
    print("전손보험사고:", driver.find_element(By.XPATH, "//th[contains(.,'전손 보험사고')]/following-sibling::td").text)
    print("침수보험사고:", driver.find_element(By.XPATH, "//th[contains(.,'침수 보험사고')]/following-sibling::td").text)
    print("도난보험사고:", driver.find_element(By.XPATH, "//th[contains(.,'도난 보험사고')]/following-sibling::td").text)

finally:
    # 6. 브라우저 닫기
    print()

일반사양: 현대아반떼 (CN7)2022년식세단 4도어
용도변경이력: 없음
번호소유자변경이력: 0회0회
특수사고이력: 전손 0회침수(전손,분손) 0회도난 0회
보험사고_내차: 없음
보험사고_타차: 없음
제작사: 
자동차명: 
연식: 
사용연료: 
배기량: 
차체형상: 
영업용대여: 
영업용일반: 
관용용도: 
전손보험사고: 
침수보험사고: 
도난보험사고: 



#### main

### 연습용매물3개 추출

In [21]:
def fetch_insurance_history_to_csv(driver, vehicle_ids, output_file="insurance_history.csv"):
    """Fetch insurance history for given vehicle IDs and save to CSV."""
    fieldnames = [
        "Vehicle ID", "Car Number", "Number Change History", "Owner Change History",
        "Total Loss History", "Flood Damage History", "Theft History",
        "Insurance Claim (My Car)", "Insurance Claim (Other Car)"
    ]
    
    data = []

    for vehicle_id in vehicle_ids:
        url = f"https://fem.encar.com/cars/report/accident/{vehicle_id}"
        driver.get(url)
        
        # Default values for missing data
        insurance_data = {
            "Vehicle ID": vehicle_id,
            "Car Number": "N/A",
            "Number Change History": "N/A",
            "Owner Change History": "N/A",
            "Total Loss History": "N/A",
            "Flood Damage History": "N/A",
            "Theft History": "N/A",
            "Insurance Claim (My Car)": "N/A",
            "Insurance Claim (Other Car)": "N/A"
        }

        try:
            # Extracting Car Number
            insurance_data["Car Number"] = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/dl/dd[1]"))
            ).text or "N/A"

            # Extracting Number Change History
            insurance_data["Number Change History"] = driver.find_element(
                By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[3]/span/span[1]"
            ).text or "N/A"

            # Extracting Owner Change History
            insurance_data["Owner Change History"] = driver.find_element(
                By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[3]/span/span[2]"
            ).text or "N/A"

            # Extracting Total Loss History
            insurance_data["Total Loss History"] = driver.find_element(
                By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[4]/span/span[1]"
            ).text or "N/A"

            # Extracting Flood Damage History
            insurance_data["Flood Damage History"] = driver.find_element(
                By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[4]/span/span[2]"
            ).text or "N/A"

            # Extracting Theft History
            insurance_data["Theft History"] = driver.find_element(
                By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[4]/span/span[3]"
            ).text or "N/A"

            # Extracting Insurance Claim (My Car)
            insurance_data["Insurance Claim (My Car)"] = driver.find_element(
                By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[5]/span"
            ).text or "N/A"

            # Extracting Insurance Claim (Other Car)
            insurance_data["Insurance Claim (Other Car)"] = driver.find_element(
                By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[6]/span"
            ).text or "N/A"

        except Exception as e:
            print(f"Error fetching data for Vehicle ID {vehicle_id}: {e}")

        data.append(insurance_data)

    # Write data to CSV
    with open(output_file, "w", newline="", encoding="utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(data)
    
    print(f"Insurance history saved to {output_file}")


# Example usage
if __name__ == "__main__":
    vehicle_ids = ["38420143", "37309395", "38019223"]  # Replace with actual IDs
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    try:
        fetch_insurance_history_to_csv(driver, vehicle_ids)
    finally:
        driver.quit()


Insurance history saved to insurance_history.csv


### 데이터 확인용 (임시)

In [25]:
import pandas as pd

# CSV 파일 읽기
df = pd.read_csv("insurance_history.csv")

# 데이터 확인
print(df.head())

   Vehicle ID Car Number Number Change History Owner Change History  \
0    38420143    19루6906                    0회                   2회   
1    37309395    97수6392                    0회                   1회   
2    38019223    45더8005                    0회                   6회   

  Total Loss History Flood Damage History Theft History  \
0              전손 0회         침수(전손,분손) 0회         도난 0회   
1              전손 0회         침수(전손,분손) 0회         도난 0회   
2              전손 0회         침수(전손,분손) 0회         도난 0회   

  Insurance Claim (My Car) Insurance Claim (Other Car)  
0             2회3,226,771원                          없음  
1                       없음                          없음  
2                       없음                          없음  


### 150개 추출 바탕으로 csv 추출

In [26]:
import csv
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# 페이지 순회를 위한 범위 지정 (예: 1~15페이지)
page_start = 1
page_end = 15  # 원하는 페이지 수까지 변경 가능
wait_time = 1  # 페이지 로딩 대기 시간

# 전체 추출 번호를 저장할 리스트
all_numbers = set()

# 크롬 드라이버 초기화
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# 1단계: 차량 번호 추출
for page in range(page_start, page_end + 1):
    try:
        # 페이지 URL 업데이트
        url = f"https://car.encar.com/list/car?page={page}"
        driver.get(url)
        
        # 페이지 로딩 대기
        time.sleep(wait_time)  # 필요에 따라 WebDriverWait으로 변경 가능
        
        # data-impression 값 추출
        index = 1
        while True:
            try:
                xpath = f"/html/body/div/div[1]/div[4]/div[1]/div[{index}]"
                element = driver.find_element(By.XPATH, xpath)
                data_impression = element.get_attribute("data-impression")
                if data_impression:
                    # 맨 앞 번호만 추출
                    first_number = data_impression.split("|")[0]
                    all_numbers.add(first_number)  # 번호 누적 저장
                index += 1
            except:
                # 현재 페이지의 매물 끝
                break
        
        print(f"페이지 {page} 크롤링 완료, 현재까지 추출된 번호 수: {len(all_numbers)}")
    except Exception as e:
        print(f"페이지 {page}에서 오류 발생: {e}")

print("모든 페이지에서 추출된 번호들:")
print(all_numbers)

# 2단계: 추출된 번호를 이용하여 보험 이력 저장
def fetch_insurance_history_to_csv(driver, vehicle_ids, output_file="insurance_history.csv"):
    """Fetch insurance history for given vehicle IDs and save to CSV."""
    fieldnames = [
        "Vehicle ID", "Car Number", "Number Change History", "Owner Change History",
        "Total Loss History", "Flood Damage History", "Theft History",
        "Insurance Claim (My Car)", "Insurance Claim (Other Car)"
    ]
    
    data = []

    for vehicle_id in vehicle_ids:
        url = f"https://fem.encar.com/cars/report/accident/{vehicle_id}"
        driver.get(url)
        
        # Default values for missing data
        insurance_data = {
            "Vehicle ID": vehicle_id,
            "Car Number": "N/A",
            "Number Change History": "N/A",
            "Owner Change History": "N/A",
            "Total Loss History": "N/A",
            "Flood Damage History": "N/A",
            "Theft History": "N/A",
            "Insurance Claim (My Car)": "N/A",
            "Insurance Claim (Other Car)": "N/A"
        }

        try:
            # Extracting Car Number
            insurance_data["Car Number"] = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/dl/dd[1]"))
            ).text or "N/A"

            # Extracting Number Change History
            insurance_data["Number Change History"] = driver.find_element(
                By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[3]/span/span[1]"
            ).text or "N/A"

            # Extracting Owner Change History
            insurance_data["Owner Change History"] = driver.find_element(
                By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[3]/span/span[2]"
            ).text or "N/A"

            # Extracting Total Loss History
            insurance_data["Total Loss History"] = driver.find_element(
                By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[4]/span/span[1]"
            ).text or "N/A"

            # Extracting Flood Damage History
            insurance_data["Flood Damage History"] = driver.find_element(
                By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[4]/span/span[2]"
            ).text or "N/A"

            # Extracting Theft History
            insurance_data["Theft History"] = driver.find_element(
                By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[4]/span/span[3]"
            ).text or "N/A"

            # Extracting Insurance Claim (My Car)
            insurance_data["Insurance Claim (My Car)"] = driver.find_element(
                By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[5]/span"
            ).text or "N/A"

            # Extracting Insurance Claim (Other Car)
            insurance_data["Insurance Claim (Other Car)"] = driver.find_element(
                By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[6]/span"
            ).text or "N/A"

        except Exception as e:
            print(f"Error fetching data for Vehicle ID {vehicle_id}: {e}")

        data.append(insurance_data)

    # Write data to CSV
    with open(output_file, "w", newline="", encoding="utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(data)
    
    print(f"Insurance history saved to {output_file}")


# Execute the pipeline
try:
    fetch_insurance_history_to_csv(driver, list(all_numbers))
finally:
    driver.quit()

페이지 1 크롤링 완료, 현재까지 추출된 번호 수: 10
페이지 2 크롤링 완료, 현재까지 추출된 번호 수: 20
페이지 3 크롤링 완료, 현재까지 추출된 번호 수: 30
페이지 4 크롤링 완료, 현재까지 추출된 번호 수: 40
페이지 5 크롤링 완료, 현재까지 추출된 번호 수: 50
페이지 6 크롤링 완료, 현재까지 추출된 번호 수: 60
페이지 7 크롤링 완료, 현재까지 추출된 번호 수: 70
페이지 8 크롤링 완료, 현재까지 추출된 번호 수: 80
페이지 9 크롤링 완료, 현재까지 추출된 번호 수: 90
페이지 10 크롤링 완료, 현재까지 추출된 번호 수: 100
페이지 11 크롤링 완료, 현재까지 추출된 번호 수: 110
페이지 12 크롤링 완료, 현재까지 추출된 번호 수: 120
페이지 13 크롤링 완료, 현재까지 추출된 번호 수: 130
페이지 14 크롤링 완료, 현재까지 추출된 번호 수: 140
페이지 15 크롤링 완료, 현재까지 추출된 번호 수: 150
모든 페이지에서 추출된 번호들:
{'38574625', '38423512', '38332191', '38487182', '38027544', '38388435', '38516529', '38577658', '38110943', '38520851', '38301461', '38039636', '38394691', '38169259', '38359313', '38282077', '38270007', '38339312', '38101168', '38203228', '38161565', '38344511', '38595187', '37919008', '38378035', '37296067', '38614209', '38425662', '38176706', '38427497', '38551389', '38412021', '38476730', '38535006', '38403767', '38601590', '38621851', '38380060', '37599137', '37746871', '38325008